In [1]:
import pybullet as p
from numpy import random
import pybullet_data


p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# First, let's make sure we start with a fresh new simulation.
# Otherwise, we can keep adding objects by running this cell over again.
p.resetSimulation()

p.setPhysicsEngineParameter(fixedTimeStep=1/100)
# Set the gravity to Earth's gravity.
p.setGravity(0, 0, -9.807)

# Define the maximum velocity limit
max_velocity_limit =2.11604775654# Adjust as needed

In [2]:
# # p.changeVisualShape(r2d2,-1,rgbaColor=[0.75,0,0,1])
# # p.changeVisualShape(r2d2,0,rgbaColor=[0.5,1,0,1])
# # p.changeVisualShape(r2d2,1,rgbaColor=[1,0.45,0.2,1])
# # p.changeVisualShape(r2d2,6,rgbaColor=[0,0.3,0.8,1])

# p.changeVisualShape(r2d2,2,rgbaColor=[1,0.5,0.8,0.89])
# # p.changeVisualShape(r2d2,4,rgbaColor=[1,0.5,0.8,0.89])

# # p.changeVisualShape(r2d2,8,rgbaColor=[1,0.5,0.8,0.89])
# # p.changeVisualShape(r2d2,10,rgbaColor=[1,0.5,0.8,0.89])

# p.changeVisualShape(r2d2,3,rgbaColor=[1,0.5,0.5,0.89])
# # p.changeVisualShape(r2d2,5,rgbaColor=[1,0.5,0.5,0.89])
# # p.changeVisualShape(r2d2,9,rgbaColor=[1,0.5,0.5,0.89])
# # p.changeVisualShape(r2d2,11,rgbaColor=[1,0.5,0.5,0.89])





# replay memory 

In [3]:
import random
from collections import namedtuple, deque
import itertools

Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward','priority'))

class ReplayMemory(object):
    def __init__(self, capacity,batch_size):
        self.memory = deque([], maxlen=capacity)
        self.sequence_length = 128  # Define the length of each sequence
        self.batch_size=batch_size
    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        sequences = []
        for _ in range(batch_size):
            while True:
                start = random.randint(0, len(self.memory) - self.sequence_length)
                sequence = list(itertools.islice(self.memory, start, start + self.sequence_length))
                # Check if sequence crosses episode boundary
                if not any(t.next_state is None for t in sequence):
                    sequences.append(sequence)
                    break
        print(len(sequences),"sequencec samples",'of lenth',len(sequences[0]))
        self.cleanup_low_reward_sequences()
        return sequences

    def calculate_sequence_reward(self, sequence):
        return sum(transition.reward for transition in sequence)

    def cleanup_low_reward_sequences(self):
        if len(self.memory) <= self.batch_size:
            return
    
        all_sequences = [list(itertools.islice(self.memory, start, start + self.sequence_length))
                         for start in range(len(self.memory) - self.sequence_length + 1)]
    
        # Compute rewards for each sequence
        sequence_rewards = [self.calculate_sequence_reward(seq) for seq in all_sequences]
    
        # Determine the number of sequences to remove (25% of sequences beyond batch size)
        num_sequences_to_remove = (len(all_sequences) - self.batch_size) // 4
    
        # Get indices of the lowest-reward sequences
        lowest_reward_indices = sorted(range(len(sequence_rewards)), key=lambda i: sequence_rewards[i])[:num_sequences_to_remove]
    
        # Remove these sequences by removing their starting transitions
        for idx in sorted(lowest_reward_indices, reverse=True):
            del self.memory[idx]

    
    def __len__(self):
        return len(self.memory)

batch_size=32
replay_memory = ReplayMemory(capacity=4096,batch_size =batch_size)  # Adjust the capacity as needed


# MODEL DEFINING

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU instead.")

Using GPU: NVIDIA GeForce GTX 1650


In [5]:
# class QuadrupedNN(nn.Module):
#     def __init__(self, input_size, output_size):
#         super(QuadrupedNN, self).__init__()
#         # Define the layers of the neural network
#         self.fc1 = nn.Linear(input_size, 128)  # First hidden layer
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 32)
#         self.fc4 = nn.Linear(32, 32)
#         self.fc5 = nn.Linear(32, output_size)

#     def forward(self, x):
#         # Forward pass through the network
#         x = F.relu(self.fc1(x))  # Activation function for first layer
#         x = F.relu(self.fc2(x))  # Activation function for second layer
#         x = F.relu(self.fc3(x))  # Activation function for second layer
#         x = F.relu(self.fc4(x))  # Activation function for second layer
#         x = self.fc5(x)          # No activation for the output layer
#         return x


# class QuadrupedLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, num_layers=2):
#         super(QuadrupedLSTM, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         # x should be of shape (batch_size, sequence_length, input_size)
#         # Initialize hidden and cell states
#         h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
#         c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)

#         # Forward propagate LSTM
#         out, _ = self.lstm(x, (h0, c0))

#         # Decode the hidden state of the last time step
#         out = self.fc(out[:, -1, :])
#         return out



# class QuadrupedLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, num_layers=2):
#         super(QuadrupedLSTM, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
#         # More fully connected layers
#         self.fc1 = nn.Linear(hidden_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         # x should be of shape (batch_size, sequence_length, input_size)
#         # Initialize hidden and cell states
#         h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
#         c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)

#         # Forward propagate LSTM
#         out, _ = self.lstm(x, (h0, c0))

#         # Decode the hidden state of the last time step
#         out = self.fc1(out[:, -1, :])
#         out = self.relu(out)
#         out = self.dropout(out)
#         out = self.fc2(out)
#         return out


class CustomSquash(nn.Module):
    def forward(self, x):
        # Squash the output to be in the range [-0.5, 0.5]
        return torch.tanh(x)

class QuadrupedTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, nhead=4, dropout=0.5):
        super(QuadrupedTransformer, self).__init__()
        
        self.embedding = nn.Linear(input_size, hidden_size)
        transformer_layer = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
        self.custom_activation = CustomSquash()

    
    def forward(self, x):
        # x should be of shape (batch_size, sequence_length, input_size)
        x = self.embedding(x) # Transform to hidden size

        # The Transformer expects input of shape (sequence_length, batch_size, hidden_size)
        x = x.permute(1, 0, 2)
        x = torch.tanh(x)

        # Forward propagate through the Transformer
        out = self.transformer_encoder(x)

        # Decode the hidden state of the last time step
        out = self.fc(out[-1, :, :])
        out = self.custom_activation(out)
        return out

# Model init


In [6]:
p.resetSimulation()

# Load our simulation floor plane at the origin (0, 0, 0).
ground = p.loadURDF('plane.urdf')

    
    
# We can check the number of bodies we have in the simulation.
p.getNumBodies()

1

In [7]:


    
def calculate_reward(position, orientation, joint_states, linear_vel, angular_vel, initial_position, initial_orientation,contact_points):
    # Constants
    HEIGHT_TARGET = 30  # Target height for the robot
    HEIGHT_WEIGHT = 0 # Weight for height reward
    ORIENTATION_WEIGHT =66.405104687  # Weight for orientation penalty
    POSITION_WEIGHT = 0.3320 # Weight for position penalty
    MOVEMENT_WEIGHT =31.45  # Weight for movement penalty
    CONTACT_POINTS_PENALTY_WEIGHT = 150.16548  # Weight for contact points reward/penalty
    CONTACT_POINTS_REWARD_WEIGHT = 168.553
    LEG_POSITION_PENALTY_WEIGHT = 64.35130  # Adjust as needed
    TORQUE_PENALTY_WEIGHT = 54.450

    x, y, z = position
    initial_x, initial_y, _ = initial_position

    # Height Reward
    height_reward = -HEIGHT_WEIGHT * abs(z - HEIGHT_TARGET)

    # Orientation Penalty
    orientation_error = np.linalg.norm(np.array(orientation) - np.array(initial_orientation))
    orientation_penalty = -ORIENTATION_WEIGHT * orientation_error

    # Position Penalty
    position_penalty = -POSITION_WEIGHT * (abs(x - initial_x) + abs(y - initial_y))

    # Movement Penalty
    joint_velocities = [state[1] for state in joint_states]
    movement_penalty = -MOVEMENT_WEIGHT * (np.linalg.norm(joint_velocities) + np.linalg.norm(linear_vel) + np.linalg.norm(angular_vel))



    contact_reward, contact_penalty= calculate_contact_reward_penalty(joint_states)

    contact_reward = CONTACT_POINTS_REWARD_WEIGHT*contact_reward
    contact_penalty = -CONTACT_POINTS_PENALTY_WEIGHT*contact_penalty
    
    joint_torques = [state[3] for state in joint_states]  # Extract joint torques
    torque_penalty = -TORQUE_PENALTY_WEIGHT * sum(abs(torque) for torque in joint_torques)


    current_joint_positions = [joint_states[idx][0] for idx in foot_joint_indices]
    leg_dev = [abs(i-j) for i in current_joint_positions for j in target_joint_positions ]
    leg_position_penalty = -LEG_POSITION_PENALTY_WEIGHT*sum(leg_dev)  # Adjust as neededcurrent_joint_positions

    # Total Reward
    total_reward = height_reward + orientation_penalty + position_penalty + movement_penalty + torque_penalty + leg_position_penalty +contact_reward + contact_penalty


    

    return total_reward


def calculate_contact_reward_penalty(joint_states,leg_joint_indices=[3,9,5,11]):
    contact_reward = 0
    contact_penalty = 0

    for joint_index in range(len(joint_states)):
        # Check for contact with the ground
        contacts = p.getContactPoints(bodyA=r2d2, bodyB=ground, linkIndexA=joint_index)

        if contacts:
            if joint_index in leg_joint_indices:
                # Reward for leg joints making contact
                contact_reward += 1
            else:
                # Penalty for other joints making contact
                contact_penalty += 1

    return contact_reward, contact_penalty


    




In [ ]:

import logging

def setup_logging():
    logging.basicConfig(filename='trakkkkining.log', level=logging.DEBUG,
                        format='%(asctime)s %(levelname)s %(processName)s: %(message)s')



def train_robot(process_id, model_queue, best_model_event):

    setup_logging()
    logging.debug(f"Starting process {process_id}")

    replay_memory = ReplayMemory(capacity=4096,batch_size =batch_size)  # Adjust the capacity as needed


    
    # Initialize PyBullet, model, and other necessary components

    p.resetSimulation()
    
    # Load an R2D2 droid at the position at 0.5 meters height in the z-axis.
    r2d2 = p.loadURDF('1/bittle.urdf',  [5*process_id, 0, 2.6],globalScaling=3.5, flags=p.URDF_USE_SELF_COLLISION)

        
    for i in range(4000):
        p.stepSimulation()


    
    # Capture initial position
    initial_position, _ = p.getBasePositionAndOrientation(r2d2)
    initial_x, initial_y, _ = initial_position



    # Observations
    position, orientation = p.getBasePositionAndOrientation(r2d2)
    x, y, z = position
    roll, pitch, yaw = p.getEulerFromQuaternion(orientation)
    joint_states = p.getJointStates(r2d2, range(p.getNumJoints(r2d2)))
    joint_positions = [state[0] for state in joint_states]  # Joint positions
    joint_velocities = [0 for state in joint_states]  # Joint velocities
    contact_points = len(p.getContactPoints(bodyA=ground, bodyB=r2d2))
    
    joint_torques = [state[3] for state in joint_states]  # Joint torques
    linear_vel, angular_vel = p.getBaseVelocity(r2d2)
    
    # Combine all observations
    observations = [x, y, z, roll, pitch, yaw] + joint_positions + joint_velocities + [contact_points] +list(linear_vel)+list(angular_vel)+list(joint_torques)
    
    
    input_size = len(observations)  # This should be the length of your observation vector
    output_size = p.getNumJoints(r2d2) 
    
    
    if best_model_event.is_set():
        model = model_queue.get()
        best_model_event.clear()
    else : 
        #model init
        model = QuadrupedTransformer(input_size,4, output_size)
        model = model.to(device)
    
    
    # Example joint_states structure: [(position, velocity, reaction_forces, applied_effort), ...]
    foot_joint_indices = [3, 5, 9, 11,2,4,8,10]
    
    target_joint_positions = [joint_states[idx][0] for idx in foot_joint_indices]
    target_joint_positions



    initial_orientation = orientation
    
    
    # Run the simulation for a fixed amount of steps.
    observations = []
    
    
    # Initialize training variables
    train = True  # Set this to True or False
    training_data = []  # To store (observation, action, reward)
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Example optimizer
    
    t=0
    tt=128
    train_alternator = -1 
    #start sim
    episode=1
    trains = 0
    while trains<15 :
        t+=1
        if t==tt:
            p.resetBasePositionAndOrientation(r2d2, initial_position, initial_orientation)
    
            for joint in  range(p.getNumJoints(r2d2)):
                p.resetJointState(r2d2, joint, targetValue=0, targetVelocity=0)
            t=0
            episode+=1
            # print(episode,":episode finised")
            continue
    
    
    
        ##OBSERVATIONS
        
        # Observations
        position, orientation = p.getBasePositionAndOrientation(r2d2)
        x, y, z = position
        roll, pitch, yaw = p.getEulerFromQuaternion(orientation)
        joint_states = p.getJointStates(r2d2, range(p.getNumJoints(r2d2)))
        joint_positions = [state[0] for state in joint_states]  # Joint positions
        joint_velocities = [state[1] for state in joint_states]  # Joint velocities
        contact_points = len(p.getContactPoints(bodyA=ground, bodyB=r2d2))
        linear_vel, angular_vel = p.getBaseVelocity(r2d2)
    
        joint_torques = [state[3] for state in joint_states]  # Joint torques
        
        # Combine all observations
        observations = [x, y, z, roll, pitch, yaw] + joint_positions + joint_velocities + [contact_points] + list(linear_vel) +list(angular_vel) +list(joint_torques)
        # print(contact_points)
        # # Convert observations to a PyTorch tensor
        # observations_tensor = torch.tensor(observations, dtype=torch.float32)
    
    
        observations_tensor = torch.tensor(observations, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
        observations_tensor = observations_tensor.to(device)
        actions = model(observations_tensor).detach().cpu().numpy()[0]
    
    
        
    
        for i in range(p.getNumJoints(r2d2)):
           
            p.setJointMotorControl2(bodyIndex=1,jointIndex=i, controlMode=p.VELOCITY_CONTROL, targetVelocity=max_velocity_limit*actions[i])
    
    
    
        
        # next Observations
        position, orientation = p.getBasePositionAndOrientation(r2d2)
        x, y, z = position
        roll, pitch, yaw = p.getEulerFromQuaternion(orientation)
        joint_states = p.getJointStates(r2d2, range(p.getNumJoints(r2d2)))
        joint_positions = [state[0] for state in joint_states]  # Joint positions
        joint_velocities = [state[1] for state in joint_states]  # Joint velocities
        contact_points = len(p.getContactPoints(bodyA=ground, bodyB=r2d2))
        linear_vel, angular_vel = p.getBaseVelocity(r2d2)
        joint_torques = [state[3] for state in joint_states]  # Joint torques
        
        # Combine all observations
        next_observations = [x, y, z, roll, pitch, yaw] + joint_positions + joint_velocities + [contact_points] + list(linear_vel) +list(angular_vel) +list(joint_torques)
    
        # # Convert observations to a PyTorch tensor
        # next_observations_tensor = torch.tensor(next_observations, dtype=torch.float32)
        
        
        reward = calculate_reward(position, orientation, joint_states, linear_vel, angular_vel, initial_position, initial_orientation,contact_points)
    
        
        # Store data for training
        if train:
            # Convert observations and next_observations to PyTorch tensors
            observations_tensor = torch.tensor(observations, dtype=torch.float32).unsqueeze(0).to(device)
            next_observations_tensor = torch.tensor(next_observations, dtype=torch.float32).unsqueeze(0).to(device)
            actions_tensor = torch.tensor(actions, dtype=torch.float32).unsqueeze(0).to(device)
            reward_tensor = torch.tensor([reward], dtype=torch.float32).to(device)
        
            # Store the transition in replay memory
        
            # Calculate initial priority for the new experience
            # Using absolute reward as a simple priority measure
            initial_priority = abs(reward)
    
            # Store the transition with priority in replay memory
            replay_memory.push(observations_tensor, actions_tensor, next_observations_tensor, reward_tensor, initial_priority)
    
    
        # Perform training 
    
        if train and len(replay_memory) >= batch_size and episode%50==0 :
            episode+=1
            trains+=1
            train_alternator=train_alternator*-1
            print('training',train_alternator)
            gamma = 0.99
            sequences = replay_memory.sample(batch_size)
            for sequence in sequences:
    
                
                batch = Transition(*zip(*sequence))
            
                # Concatenate the batch elements into separate tensors
                state_batch = torch.cat(batch.state).unsqueeze(1)
                action_batch = torch.cat(batch.action)
                reward_batch = torch.cat(batch.reward)
                next_state_batch = torch.cat(batch.next_state).unsqueeze(1)
            
                # Predict current Q-values
                # print(state_batch.shape,reward_batch.shape)
                current_q_values = model(state_batch)
        
        
                
            
                # Predict next Q-values
                next_q_values = model(next_state_batch)
            
                # Average next Q-values across joints
                average_next_q_values = torch.mean(next_q_values, dim=1)
            
                # Calculate TD target
                td_target = reward_batch + gamma * average_next_q_values
            
                # Compute loss
                # print(current_q_values)
                loss = F.mse_loss(current_q_values, td_target.unsqueeze(1))
        
            
                # Perform optimization step
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
    
        p.stepSimulation()
    
            

    # Evaluate the model
    performance = reward

    # If this is the best model so far, share it with other processes
    with model_lock:
        if performance > shared_performance.value:
            shared_performance.value = performance
            model_queue.put(model)
            best_model_event.set()


In [ ]:
from multiprocessing import Process, Queue, Event, Value, Lock
import multiprocessing

# Shared components
model_queue = Queue()
best_model_event = Event()
shared_performance = Value('d', -float('inf'))  # Shared variable for best performance
model_lock = Lock()  # Lock for updating shared performance

# Number of robots
num_robots = 4

processes = []
num_robots=1
for i in range(num_robots):
    q = Process(target=train_robot, args=(i, model_queue, best_model_event))
    q.start()
    processes.append(q)


In [ ]:
for p in processes:
    p.join()
    print(f"Process {p.name} alive: {p.is_alive()}")

In [ ]:
model_queue.get()